In [10]:
import os
os.environ["LABEL_STUDIO_LOCAL_FILES_DOCUMENT_ROOT"] = "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data"

import sys
import requests
from os.path import join, exists, basename
from glob import glob
from label_studio_sdk import Client
from copy import deepcopy

import torch
import numpy as np
from ultralytics.utils.ops import xyxyxyxy2xywhr, xywhr2xyxyxyxy
from ultralytics.utils.metrics import probiou
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import warnings

from joblib import Parallel, delayed

sys.path.append("/home/patel_zeel/kilns_neurips24/scripts")

In [11]:
from utils import yolo_obb_to_label_studio_json

## Label Studio Initialize

In [12]:
LABEL_STUDIO_URL = 'http://10.0.62.168:8082'
API_KEY = '2557d98fa3032fc9a2eb033baf2775ebc2e1a0ef'

ls = Client(url=LABEL_STUDIO_URL, api_key=API_KEY)

## Process the input data

In [9]:
pred_txt_path = "../data/predict/trench_width_delhi_ncr_obb_v1_640_100/delhi_ncr_data/predict_v1/labels"
image_read_dir = "../data/predict/trench_width_delhi_ncr_obb_v1_640_100/delhi_ncr_data/predict_v1/images"

state = "delhi_ncr"
version = "250"
score_threshold = 0.25
ls_project_name = f"{state}_{version}_{version}"
save_dir = f"/home/patel_zeel/kilns_neurips24/pred_images/{state}_{version}"
corrected_txt_path = join("/home/patel_zeel/compass24/corrected_txt/", state)


pred_files = sorted(glob(join(pred_txt_path, state, "labels", "*.txt")))
pred_base_names = set([basename(f) for f in pred_files])
corrected_files = sorted(glob(join(corrected_txt_path, "*.txt")))
corrected_base_names = set([basename(f) for f in corrected_files])

print(len(pred_files), len(corrected_files))

16921 7183


In [10]:
common_tiles = []
uncommon_tiles = []
for pred_base, pred_full in tqdm(zip(pred_base_names, pred_files)):
    if pred_base in corrected_base_names:
        common_tiles.append(pred_base)
    else:
        uncommon_tiles.append(pred_full)
len(common_tiles), len(uncommon_tiles)

0it [00:00, ?it/s]

(6491, 10430)

Remove lucknow files

In [11]:
lucknow_files = glob(join("/home/patel_zeel/kilns_neurips24/processed_data/lucknow_sarath_grid_obb_v3/labels", "*.txt"))
lucknow_base_names = set([basename(f) for f in lucknow_files])

lucknow_removed_uncommon = []
for full in tqdm(uncommon_tiles):
    base = basename(full)
    if base in lucknow_base_names:
        pass
    else:
        lucknow_removed_uncommon.append(full)
    
len(lucknow_removed_uncommon)

  0%|          | 0/10430 [00:00<?, ?it/s]

10264

## Filter further

In [17]:
set_all_till_now = set(map(basename, glob("/home/patel_zeel/kilns_neurips24/corrected_txt_obb/uttar_pradesh/*.txt")))
print(len(set_all_till_now))
print(len(lucknow_removed_uncommon))
filtered_uncommon = [file for file in lucknow_removed_uncommon if basename(file) not in set_all_till_now]
len(filtered_uncommon)

9887
10264


5049

In [ ]:
# Save symlink of files
save_image_dir = f"/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/predict/trench_width_delhi_ncr_obb_v1_640_100/delhi_ncr_data/predict_v1/images"
save_label_dir = f"/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/predict/trench_width_delhi_ncr_obb_v1_640_100/delhi_ncr_data/predict_v1/labels"
print("saving to", save_image_dir, save_label_dir)
os.system("rm -rf " + save_image_dir)
os.system("rm -rf " + save_label_dir)
os.makedirs(save_image_dir)
os.makedirs(save_label_dir)

for tile in tqdm(filtered_uncommon):
    part = tile.split("/")[-3]
    name = tile.split("/")[-1].replace(".txt", ".png")
    image_path = join(image_read_dir, part, name)
    txt_path = join(pred_txt_path, part, "labels", name.replace(".png", ".txt"))
    os.symlink(image_path, join(save_image_dir, name))
    os.symlink(txt_path, join(save_label_dir, name.replace(".png", ".txt")))

print("image_path", image_path)
print("txt_path", txt_path)
print(join(save_image_dir, name))
print(join(save_label_dir, name.replace(".png", ".txt")))

saving to /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/predict/trench_width_delhi_ncr_obb_v1_640_100/delhi_ncr_data/predict_v1/images /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/predict/trench_width_delhi_ncr_obb_v1_640_100/delhi_ncr_data/predict_v1/labels


NameError: name 'filtered_uncommon' is not defined

## Create Project

In [14]:
state = "delhi_ncr"
version = "250"

In [15]:
projects = ls.list_projects()
for project in projects:
    if project.title == f"{state}_OBB_{version}":
        print("Project found. Using it...")
        break
else:
    print("No project found. Creating...")
    project = ls.start_project(title=f"{state}_OBB_{version}", label_config='''
<View>
  <Image name="image" value="$image" rotateControl="true" zoomControl="true" zoom="true"/>
  <RectangleLabels name="label" toName="image">
    
    
  <Label value="Inner_Box" background="#FFA39E"/><Label value="Outer_Box" background="#63c205"/></RectangleLabels>
</View>
    ''')
storages = project.get_import_storages()
if len(storages) == 0:
    print("No storage found. Creating...")
    project.connect_local_import_storage(save_image_dir, regex_filter=".*png")
else:
    assert len(storages) == 1
    print("Storage found. Using it...")
storage = storages[0]
if storage['last_sync'] is None:
    print("Syncing storage...")
    project.sync_storage(storage['type'], storage['id'])
else:
    print("Storage already synced.")

print("Getting tasks...")
tasks = project.get_tasks()
len(tasks)

No project found. Creating...
No storage found. Creating...


NameError: name 'save_image_dir' is not defined

In [21]:
tasks[0]

{'id': 183999,
 'predictions': [],
 'annotations': [],
 'drafts': [],
 'annotators': [],
 'inner_id': 1,
 'cancelled_annotations': 0,
 'total_annotations': 0,
 'total_predictions': 0,
 'completed_at': None,
 'annotations_results': '',
 'predictions_results': '',
 'predictions_score': None,
 'file_upload': None,
 'storage_filename': '/home/patel_zeel/kilns_neurips24/pred_images/uttar_pradesh_v2/images/23.93,82.91.png',
 'annotations_ids': '',
 'predictions_model_versions': '',
 'avg_lead_time': None,
 'draft_exists': False,
 'updated_by': [],
 'data': {'image': '/data/local-files/?d=home/patel_zeel/kilns_neurips24/pred_images/uttar_pradesh_v2/images/23.93%2C82.91.png'},
 'meta': {},
 'created_at': '2024-06-04T06:27:51.726793Z',
 'updated_at': '2024-06-04T06:27:51.726815Z',
 'is_labeled': False,
 'overlap': 1,
 'comment_count': 0,
 'unresolved_comment_count': 0,
 'last_comment_updated_at': None,
 'project': 63,
 'comment_authors': []}

In [22]:
## Delete all predictions before going ahead
os.system(f"curl -H 'Authorization: Token {API_KEY}' -X POST \"localhost:8080/api/dm/actions?id=delete_tasks_predictions&project={project.id}\"")

for task in tqdm(tasks):
    # part = tile.split("/")[-3]
    # name = tile.split("/")[-1].replace(".txt", ".png")
    # image_path = join(save_dir, name)
    image_path = task['storage_filename']
    txt_name = basename(image_path).replace(".png", ".txt")
    
    preds = np.loadtxt(join(save_label_dir, txt_name))
    preds = np.atleast_2d(preds)

    results = []
    for pred in preds:
        label_id = int(pred[0])
        x1, y1, x2, y2, x3, y3, x4, y4 = pred[1:-1]
        score = pred[-1]
        if score > score_threshold:
          prediction = yolo_obb_to_label_studio_json(label_id, x1, y1, x2, y2, x3, y3, x4, y4, score)
          # predictions.append(prediction)
          results.append(prediction['result'][0])
    if len(results) > 0:
      # I have tried parallelizing this but it doesn't work
      project.create_prediction(task['id'], results, model_version=version)

{"processed_items":0,"detail":"Deleted 0 predictions"}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    54  100    54    0     0   1636      0 --:--:-- --:--:-- --:--:--  1636


  0%|          | 0/5049 [00:00<?, ?it/s]

## Appendix

In [ ]:
# areas = []
# for item in json_list:
#     for pred in item["predictions"]:
#         area = pred['result'][0]['value']['width'] * pred['result'][0]['value']['height']
#         area = area * (1120 * 1120) / (100 * 100)
#         areas.append(area)
# len(areas)

In [ ]:
# max(areas), min(areas), np.mean(areas), np.median(areas)

In [ ]:
# import seaborn as sns

# sns.kdeplot(areas)

In [ ]:
# (areas < np.mean(areas) - 2 * np.std(areas)).sum()